- csdf_cs_dnac_appliances
- cdna_appliances

In [1]:
import pandas as pd
import os
from tqdm import tqdm

In [2]:
pwd

'/home/jupyter/sw_xray_analysis/telemetry'

In [3]:
cr_party_name = pd.read_csv('/home/jupyter/DATA/input/telemetry_analysis/cr_party_name.csv', index_col=0)
sr_customer_name = pd.read_csv('/home/jupyter/DATA/input/telemetry_analysis/sr_customer_name.csv', index_col=0)

In [4]:
sr_customer_name

,customer_gu_name
0,1-800-FLOWERSCOM INC
1,1199 SEIU LEAGUE JOB SECURITY FUND
2,3M COMPANY
3,3MD TECHNOLOGIES INC
4,"8X8, INC."
...,...
3614,ZOLIONDA GROUP
3615,ZUHLKE TECHNOLOGY GROUP AG
3616,ZURCHER HOCHSCHULE FUR ANGEWANDTE WISSENSCHAFT...
3617,ZURICH INSURANCE GROUP AG


In [5]:
cr_party_name

,cr_party_name
0,1-800-FLOWERS RETAIL INC
1,1199SEIU FUNDS
2,16TH CIRCUIT COURT OF JACKSON COUNTY
3,3-BANKEN BETEILIGUNG GMBH
4,39 IOS
...,...
4874,ZOETIS EXTON
4875,ZOLIONDA GROUP
4876,ZUHLKE ENGINEERING AG
4877,ZURCHER HOCHSCHULE FUR ANGEWANDTE WISSENSCHAFT...


# Company name matching
## Play around with `fuzzywuzzy` package

Ref 1: https://github.com/seatgeek/fuzzywuzzy     
Ref 2: http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/

Fuzzy string matching uses [Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) Distance to calculate the differences between sequence.



In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
# Exact match, the Levenshtein Distance of two string. 
fuzz.ratio("California", "California State")

77

In [8]:
# The ratio of most similar substring.
fuzz.partial_ratio("California", "California State")

100

In [9]:
# Measure of the sequences' similarity sorting the token before comparing.
# tokenizing the string in question, sorting the tokens alphabetically, and then joining them back into a string.
fuzz.token_sort_ratio("State California", "California State")

100

In [10]:
# Find all alphanumeric tokens in each string.
# tokenize both strings, but instead of immediately sorting and comparing, we split the tokens into two groups: intersection and remainder. We use those sets to build up a comparison string.
fuzz.token_set_ratio("State California", "State  State California")

100

In [11]:
fuzz.token_set_ratio("1-800-FLOWERSCOM INC", "1-800-FLOWERS RETAIL INC")

77

In [12]:
choices = ["California", "Los Angeles", "San Jose", "San Bernardino"]
process.extract("L.A", choices, limit=2)


[('Los Angeles', 60), ('California', 57)]

In [13]:
process.extractOne("L.A", choices)

('Los Angeles', 60)

## Implement fuzzywuzzy on our data

In [14]:
choices = cr_party_name['cr_party_name'].tolist()
process.extract("1-800-FLOWERSCOM INC", choices, limit=2, scorer=fuzz.token_set_ratio) 

[('1-800-FLOWERS RETAIL INC', 77), ('ACT, INC', 60)]

In [54]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold = 75, limit =3, scorer = fuzz.token_set_ratio):
    
    """
    df_1: left table to join
    df_2: right table to join 
    key1: key column of left table 
    key2: key column of right table 
    threshold: how close the matchs should be return to match, based on Levenshtein distance
    limit: amount of matches will return
    scorer: scorer will be used in matching
    return: dataframe with both keys and matches
    """
    
    choice_list = df_2[key2].tolist()
    
    m1 = df_1[key1].apply(lambda x: process.extract(x, choice_list, limit = limit, scorer = scorer))
    df_1['best matches'] = m1
    
    m2 = df_1['best matches'].apply(lambda x:  [i[0] for i in x if i[1] >= threshold][0] 
                                        if len([i[0] for i in x if i[1] >= threshold]) > 0 
                                        else '')

    df_1['top match'] = m2
    
    
    
    return df_1


In [57]:
# run time: 25mins
df = fuzzy_merge(sr_customer_name, 
                 cr_party_name, 
                 'customer_gu_name', 
                 'cr_party_name', 
                 threshold = 75
                )


In [58]:
df

,customer_gu_name,best matches,top match
0,1-800-FLOWERSCOM INC,"[(1-800-FLOWERS RETAIL INC, 77), (ACT, INC, 60), (ADP INC, 60)]",1-800-FLOWERS RETAIL INC
1,1199 SEIU LEAGUE JOB SECURITY FUND,"[(STATE SECURITY AGENCY, 55), (SECURITAS NORGE AS, 54), (SECURITIES AND EXCHANGE COMMISSION US, 54)]",
2,3M COMPANY,"[(3M COMPANY, 100), (AA ACQUISITION COMPANY LTD, 82), (ABU DHABI AIRPORTS COMPANY, 82)]",3M COMPANY
3,3MD TECHNOLOGIES INC,"[(AGILENT TECHNOLOGIES INC US, 89), (FORESCOUT TECHNOLOGIES INC, 89), (KEYSIGHT TECHNOLOGIES, INC., 89)]",AGILENT TECHNOLOGIES INC US
4,"8X8, INC.","[(NUTANIX C O 8X8 INC, 100), (KOFAX INC, 62), (1-800-FLOWERS RETAIL INC, 60)]",NUTANIX C O 8X8 INC
...,...,...,...
3614,ZOLIONDA GROUP,"[(ZOLIONDA GROUP, 100), (NEC GROUP, 71), (SGH GROUP, 71)]",ZOLIONDA GROUP
3615,ZUHLKE TECHNOLOGY GROUP AG,"[(MILLENNIUM TECHNOLOGY GROUP, 76), (TECHNOLOGY GROUP SOLUTIONS LLC, 76), (SMA SOLAR TECHNOLOGY AG, 72)]",MILLENNIUM TECHNOLOGY GROUP
3616,ZURCHER HOCHSCHULE FUR ANGEWANDTE WISSENSCHAFTEN ZHAW,"[(ZURCHER HOCHSCHULE FUR ANGEWANDTE WISSENSCHAFTEN ZHAW, 100), (PAEDAGOGISCHE HOCHSCHULE ZURICH, 57), (FRAUNHOFER-GESELLSCHAFT ZUR FOERDERUNG DER ANGEWANDTEN FORSCHUNG EV, 53)]",ZURCHER HOCHSCHULE FUR ANGEWANDTE WISSENSCHAFTEN ZHAW
3617,ZURICH INSURANCE GROUP AG,"[(AG INSURANCE SA, 89), (DB INSURANCE, 86), (EMC INSURANCE GROUP INC., 79)]",AG INSURANCE SA


In [59]:
# Display full (no truncate) dataframe

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', -1)

In [60]:
df.head(100)

,customer_gu_name,best matches,top match
0,1-800-FLOWERSCOM INC,"[(1-800-FLOWERS RETAIL INC, 77), (ACT, INC, 60), (ADP INC, 60)]",1-800-FLOWERS RETAIL INC
1,1199 SEIU LEAGUE JOB SECURITY FUND,"[(STATE SECURITY AGENCY, 55), (SECURITAS NORGE AS, 54), (SECURITIES AND EXCHANGE COMMISSION US, 54)]",
2,3M COMPANY,"[(3M COMPANY, 100), (AA ACQUISITION COMPANY LTD, 82), (ABU DHABI AIRPORTS COMPANY, 82)]",3M COMPANY
3,3MD TECHNOLOGIES INC,"[(AGILENT TECHNOLOGIES INC US, 89), (FORESCOUT TECHNOLOGIES INC, 89), (KEYSIGHT TECHNOLOGIES, INC., 89)]",AGILENT TECHNOLOGIES INC US
4,"8X8, INC.","[(NUTANIX C O 8X8 INC, 100), (KOFAX INC, 62), (1-800-FLOWERS RETAIL INC, 60)]",NUTANIX C O 8X8 INC
5,?BANK OZK,"[(?BANK OZK, 100), (MT BANK, 73), (TD BANK, 73)]",?BANK OZK
6,A LOACKER SPA,"[(A LOACKER SPA, 100), (ENI SPA, 60), (ROCHE FARMA SA, 59)]",A LOACKER SPA
7,A-LEHDET OY,"[(A-LEHDET OY, 100), (VIESTILEHDET OY, 69), (MEDBIT OY, 60)]",A-LEHDET OY
8,A. T. STILL UNIVERSITY OF HEALTH SCIENCES,"[(A. T. STILL UNIVERSITY OF HEALTH SCIENCES, 100), (UNIVERSITY OF IOWA, 84), (UNIVERSITY OF UTAH, 84)]",A. T. STILL UNIVERSITY OF HEALTH SCIENCES
9,AABENRAA KOMMUNE,"[(AABENRAA KOMMUNE, 100), (AALBORG KOMMUNE, 77), (FAXE KOMMUNE, 74)]",AABENRAA KOMMUNE


In [241]:
choices = cr_party_name['cr_party_name'].tolist()
process.extract("AARON'S INC", choices, limit=5, scorer=fuzz.token_set_ratio) 

[('ARINC INC', 70),
 ('ITRON INC', 70),
 ('JSC CROC INC', 70),
 ('MARCO INC', 70),
 ('AMAZONCOM INC', 67)]